<a href="https://colab.research.google.com/github/Raylynd6299/Intro-to-Learn-DRL/blob/main/Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Policy Evaluation 

In [1]:
import numpy as np
import tensorflow as tf


In [2]:
P= {
    0: {   #State- Example state 0
        0: [(1.0,0,0.0, True)], #Action (Action 0) : Possible transitions for that (State, Action)
        1: [(1.0,0,0.0, True)]  #[probabilit of that transition, next state, reward, flag thath indicated if the next state is terminal]
    },
    1: {
        0: [(1.0,0,0.0, True)],
        1: [(1.0,2,1.0, True)]
    },
    2: {
        0: [(1.0,2,0.0, True)],
        1: [(1.0,2,0.0, True)]
    }
}

In [5]:
def policy_evaluation( pi, P, gamma=1.0, theta=1e-10 ):
  pre_V = np.zeros(len(P))
  while True:
    V = np.zeros(len(P))
    for s in range(len(P)):
      for prob, next_state, reward, done in P[s][pi(s)]:
        V[s] += prob * ( reward + gamma * pre_V[next_state] * (not done));
    if np.max(np.abs(pre_V - V)) < theta:
      break
    pre_V = V.copy()
  return pre_V

In [4]:
def policy_improvement(V, P, gamma=1.0):
  Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
  for s in range(len(P)):
    for a in range(len(P[s])):
      for prob, next_state, reward, done in P[s][a]:
        Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
  new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
  return new_pi

In [6]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
  random_actions = np.random.choice(tuple(P[0].keys()), len(P))
  pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
  while True:
    old_pi = {s:pi(s) for s in range(len(P))}
    V = policy_evaluation(pi, P, gamma, theta)
    pi = policy_improvement(V, P, gamma)
    if old_pi == {s:pi(s) for s in range(len(P))}:
      break
  return V, pi

In [7]:
def value_iteration(P, gamma=1.0, theta=1e-10):
  V = np.zeros(len(P), dtype=np.float64)
  while True:
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
      for a in range(len(P[s])):
        for prob, next_state, reward, done in P[s][a]:
          Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
    if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
      break
    V = np.max(Q, axis=1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
  return V, pi